# Creating the sample_collab.db SQLite database

To be run a single time to set prefered default sqlite3 output mode (here "box" type output, with column headers):

In [15]:
echo -e '.mode box\n.headers on\n' > ~/.sqliterc

## Create the sqlite DB from original raw data files

In [ ]:
# Change the path below to match your folder's path:
cd /shared/projects/form_2022_19/==your_firstname==/sars2copath
git status
# if this does NOT return "nothing to commit, working directory clean", then you must first stage & commit your current local branch modifications:
# git add .
# git commit -m "some message"

In [24]:
git branch
git branch sample_collab_db_creation
git checkout sample_collab_db_creation
cd ./analyses
ls -ltr
ls -ltr ../data

M	analyses/sample_collab.db
Switched to branch 'sample_collab_db_creation'
total 136
-rw-r-----+ 1 phingamp phingamp 126976 May 20 00:28 sample_collab.db
-rw-rw----+ 1 phingamp phingamp   5914 May 20 00:34 sample_collab_db_schema.sql
total 740
-rw-rw----+ 1 phingamp phingamp 214743 May 13 20:48 SraRunTable.csv
-rw-rw----+ 1 phingamp phingamp  31441 May 19 13:39 sample_overviews 5.csv
-rw-rw----+ 1 phingamp phingamp  31719 May 19 13:39 sample_overviews 3.csv
-rw-rw----+ 1 phingamp phingamp  15884 May 19 13:39 ct_values_covid19_metagenomics.xlsx
-rw-rw----+ 1 phingamp phingamp 447419 May 19 13:39 filereport_read_run_PRJEB47870.tsv


## Don't forget at the end of these sandbox explorations to quit this git branch and return to your real development branch (see instructions at the end)

In [25]:
[ -f sample_collab.db ] && rm sample_collab.db
sqlite3 -batch  sample_collab.db < sample_collab_db_schema.sql

In [26]:
sed -E "s/,P158cDNA_L001,/,P158_cDNA_L001,/" ../data/SraRunTable.csv | sed -E "s/,P011_S39_L001,/,P011_DNA_S39_L001,/" > ../data/SraRunTable_mended.csv
sqlite3 -batch sample_collab.db ".import --csv --skip 1 ../data/SraRunTable_mended.csv SraRunTable"
#sqlite3 -batch sample_collab.db ".schema SraRunTable"
sqlite3 -batch sample_collab.db "select count(*) from SraRunTable"

┌──────────┐
│ count(*) │
├──────────┤
│ 250      │
└──────────┘


In [27]:
sqlite3 -batch -tabs sample_collab.db ".import --skip 1 ../data/filereport_read_run_PRJEB47870.tsv ENARunTable"
#sqlite3 -batch sample_collab.db ".schema ENARunTable"
sqlite3 -batch sample_collab.db "select count(*) from ENARunTable"

┌──────────┐
│ count(*) │
├──────────┤
│ 250      │
└──────────┘


In [28]:
cat "../data/sample_overviews 5.csv" > ../data/sample_overviews_all.csv
tail -n +2 "../data/sample_overviews 3.csv" >> ../data/sample_overviews_all.csv
wc -l ../data/sample_overviews_all.csv
sed -E "s/^P158cDNA_L001,/P158_cDNA_L001,/" ../data/sample_overviews_all.csv | sed -E "s/^P011_S39_L001,/P011_DNA_S39_L001,/" > ../data/sample_overviews_all_mended.csv
tail -n +2 ../data/sample_overviews_all_mended.csv | cut -f1 -d"," | sed -E "s/^(\S+)_(DNA|cDNA|CDNA|RNA|dna).*$/,\1/" | sed -E "s/P07_gg/P07/" | sed -E "s/^,P0+/,P/" > ../data/sample_overviews_all_mended_patient_code.csv
echo ",patient_code" | cat - ../data/sample_overviews_all_mended_patient_code.csv > ../data/sample_overviews_all_mended_patient_code_with_header.csv
paste ../data/sample_overviews_all_mended.csv ../data/sample_overviews_all_mended_patient_code_with_header.csv > ../data/sample_overviews_all_mended_patient_code_with_header_pasted.csv
sqlite3 -batch -csv sample_collab.db ".import --csv --skip 1 ../data/sample_overviews_all_mended_patient_code_with_header_pasted.csv SampleOverviews"
#sqlite3 -batch sample_collab.db ".schema SampleOverviews"
sqlite3 -batch sample_collab.db "select count(*) from SampleOverviews"

251 ../data/sample_overviews_all.csv
┌──────────┐
│ count(*) │
├──────────┤
│ 250      │
└──────────┘


In [29]:
module load csvkit
in2csv ../data/ct_values_covid19_metagenomics.xlsx > ../data/ct_values_covid19_metagenomics.csv

In [30]:
sqlite3 -batch sample_collab.db ".import --csv --skip 1 ../data/ct_values_covid19_metagenomics.csv CtValues"
#sqlite3 -batch sample_collab.db ".schema CtValues"
sqlite3 -batch sample_collab.db "select count(*) from CtValues"

┌──────────┐
│ count(*) │
├──────────┤
│ 379      │
└──────────┘


## 😉 Warm up with an easy join between TWO tables

In [31]:
sqlite3 -batch sample_collab.db \
"select count(*) \
from SraRunTable sra inner join SampleOverviews spl on sra.host_subject_id=spl.sample_name;"

┌──────────┐
│ count(*) │
├──────────┤
│ 250      │
└──────────┘


Quality control: Check that sample names are coherent between NCBI SRA and EBI ENA:

In [32]:
sqlite3 -batch sample_collab.db \
"select count(*) from  SampleOverviews spl left join SraRunTable sra on sra.host_subject_id=spl.sample_name;"
sqlite3 -batch sample_collab.db \
"select count(*) from SraRunTable sra left join SampleOverviews spl on sra.host_subject_id=spl.sample_name;"

┌──────────┐
│ count(*) │
├──────────┤
│ 250      │
└──────────┘
┌──────────┐
│ count(*) │
├──────────┤
│ 250      │
└──────────┘


## 🤨 Let's go for a DOUBLE join between THREE tables

In [33]:
sqlite3 -batch sample_collab.db "select count(*) \
from (SraRunTable sra inner join SampleOverviews spl on sra.host_subject_id=spl.sample_name) \
inner join ENARunTable ena on sra.Run=ena.run_accession;"

┌──────────┐
│ count(*) │
├──────────┤
│ 250      │
└──────────┘


## 😲 Take a deep breath : now for the TRIPLE join of the FOUR tables:

In [34]:
sqlite3 -batch sample_collab.db \
"select count(*) \
from ((SraRunTable sra inner join SampleOverviews spl on sra.host_subject_id=spl.sample_name) \
inner join ENARunTable ena on sra.Run=ena.run_accession) \
left join CtValues ct on patient_code=ct.Blinded_number;"

┌──────────┐
│ count(*) │
├──────────┤
│ 250      │
└──────────┘


## Create a couple of views to simplify usage of the sample annotations

In [35]:
sqlite3 -batch sample_collab.db \
"create view sample_annot_all as select * \
from ((SraRunTable sra inner join SampleOverviews spl on sra.host_subject_id=spl.sample_name) \
inner join ENARunTable ena on sra.Run=ena.run_accession) \
left join CtValues ct on patient_code=ct.Blinded_number;"

In [36]:
sqlite3 -batch sample_collab.db \
"create view sample_annot as select run_accession,host_subject_id,patient_code,nucleotide_type as nuc,host_body_site,\
    host_disease_status,miscellaneous_parameter,Ct,total_reads,read_count,base_count  \
from ((SraRunTable sra inner join SampleOverviews spl on sra.host_subject_id=spl.sample_name) \
inner join ENARunTable ena on sra.Run=ena.run_accession) \
left join CtValues ct on patient_code=ct.Blinded_number;"

### From now on we can just use the "sample_annot" view as if it was an ordinary table

In [37]:
sqlite3 -batch sample_collab.db \
"select * from sample_annot limit 20;"

┌───────────────┬───────────────────┬──────────────┬─────┬────────────────┬─────────────────────┬────────────────────────────┬───────┬─────────────┬────────────┬────────────┐
│ run_accession │  host_subject_id  │ patient_code │ nuc │ host_body_site │ host_disease_status │  miscellaneous_parameter   │  Ct   │ total_reads │ read_count │ base_count │
├───────────────┼───────────────────┼──────────────┼─────┼────────────────┼─────────────────────┼────────────────────────────┼───────┼─────────────┼────────────┼────────────┤
│ ERR6913102    │ P011_DNA_S39_L001 │ P11          │ DNA │ Nasopharyngeal │ SARS-CoV-2 positive │ Symptomatic-DNA-sequenced  │ 23.59 │ 27630988    │ 102524     │ 13193813   │
│ ERR6913103    │ P019_DNA_S74_L001 │ P19          │ DNA │ Nasopharyngeal │ SARS-CoV-2 positive │ Symptomatic-DNA-sequenced  │ 18.81 │ 21364260    │ 385740     │ 49060845   │
│ ERR6913108    │ P069_DNA_S27_L001 │ P69          │ DNA │ Nasopharyngeal │ SARS-CoV-2 positive │ Symptomatic-DNA-sequenced  

In [38]:
sqlite3 -batch sample_collab.db \
"select run_accession,submitted_ftp from sample_annot_all limit 20;"

┌───────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ run_accession │                                                                                    submitted_ftp                                                                                    │
├───────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ ERR6913102    │ ftp.sra.ebi.ac.uk/vol1/run/ERR691/ERR6913102/P011_S39_L001_1_68215_reads_nh_R1.fastq.gz;ftp.sra.ebi.ac.uk/vol1/run/ERR691/ERR6913102/P011_S39_L001_1_68215_reads_nh_R2.fastq.gz     │
│ ERR6913103    │ ftp.sra.ebi.ac.uk/vol1/run/ERR691/ERR6913103/P019_DNA_S74_L001_68214_reads_nh_R1.fastq.gz;ftp.sra.ebi.ac.uk/vol1/run/ERR691/ERR6913103/P019_DNA_S74_L001_68214_reads_nh_R2.fastq.gz │


## More quality controling

In [39]:
sqlite3 -batch sample_collab.db \
"select count(distinct(patient_code)) \
from ((SraRunTable sra inner join SampleOverviews spl on sra.host_subject_id=spl.sample_name) \
inner join ENARunTable ena on sra.Run=ena.run_accession) \
left join CtValues ct on patient_code=ct.Blinded_number;"

┌───────────────────────────────┐
│ count(distinct(patient_code)) │
├───────────────────────────────┤
│ 120                           │
└───────────────────────────────┘


In [40]:
sqlite3 -batch sample_collab.db \
"select count(distinct(patient_code)) \
from ((SraRunTable sra inner join SampleOverviews spl on sra.host_subject_id=spl.sample_name) \
inner join ENARunTable ena on sra.Run=ena.run_accession) \
left join CtValues ct on patient_code=ct.Blinded_number \
where patient_code like 'P%';"

┌───────────────────────────────┐
│ count(distinct(patient_code)) │
├───────────────────────────────┤
│ 104                           │
└───────────────────────────────┘


In [41]:
sqlite3 -batch sample_collab.db \
"select patient_code,host_subject_id,count(*) as nb \
from ((SraRunTable sra inner join SampleOverviews spl on sra.host_subject_id=spl.sample_name) \
inner join ENARunTable ena on sra.Run=ena.run_accession) \
left join CtValues ct on patient_code=ct.Blinded_number \
group by patient_code order by nb desc limit 20;"

┌──────────────┬───────────────────┬────┐
│ patient_code │  host_subject_id  │ nb │
├──────────────┼───────────────────┼────┤
│ P11          │ P011_DNA_S39_L001 │ 4  │
│ P19          │ P019_DNA_S74_L001 │ 4  │
│ P22          │ P022_DNA_L001     │ 4  │
│ P35          │ P035_DNA_S21_L001 │ 4  │
│ P4           │ P004_DNA_L001     │ 4  │
│ NP_A         │ NP_A_DNA_L001     │ 2  │
│ NP_B         │ NP_B_DNA_L001     │ 2  │
│ NP_C         │ NP_C_DNA_L001     │ 2  │
│ NP_D         │ NP_D_DNA_L001     │ 2  │
│ NP_E         │ NP_E_DNA_L001     │ 2  │
│ P1           │ P01_DNA_L001_2    │ 2  │
│ P10          │ P10_DNA_L001_2    │ 2  │
│ P105         │ P105_DNA_L001_1   │ 2  │
│ P113         │ P113_DNA_L001     │ 2  │
│ P12          │ P12_DNA_L001_2    │ 2  │
│ P121         │ P121_DNA_L001     │ 2  │
│ P122         │ P122_DNA_L001     │ 2  │
│ P13          │ P13_DNA_L001_2    │ 2  │
│ P133         │ P133_DNA_L001     │ 2  │
│ P138         │ P138_DNA_L001     │ 2  │
└──────────────┴──────────────────

In [42]:
sqlite3 -batch sample_collab.db \
"select host_subject_id \
from ((SraRunTable sra inner join SampleOverviews spl on sra.host_subject_id=spl.sample_name) \
inner join ENARunTable ena on sra.Run=ena.run_accession) \
left join CtValues ct on patient_code=ct.Blinded_number \
where patient_code='P11';"

┌───────────────────┐
│  host_subject_id  │
├───────────────────┤
│ P011_DNA_S39_L001 │
│ P11_DNA_L001_2    │
│ P011_RNA_S43_L001 │
│ P11_CDNA_L001_2   │
└───────────────────┘


In [43]:
sqlite3 -batch sample_collab.db \
"select host_subject_id,patient_code,nucleotide_type as nuc,total_reads,nonhost_reads,Bases,host_body_site,\
    host_disease_status,miscellaneous_parameter,Ct, spl.sample_name, read_count,base_count \
from ((SraRunTable sra inner join SampleOverviews spl on sra.host_subject_id=spl.sample_name) \
inner join ENARunTable ena on sra.Run=ena.run_accession) \
left join CtValues ct on patient_code=ct.Blinded_number \
order by host_subject_id asc limit 20;"

┌───────────────────┬──────────────┬─────┬─────────────┬───────────────┬───────────┬────────────────┬─────────────────────┬─────────────────────────────┬───────┬───────────────────┬────────────┬────────────┐
│  host_subject_id  │ patient_code │ nuc │ total_reads │ nonhost_reads │   Bases   │ host_body_site │ host_disease_status │   miscellaneous_parameter   │  Ct   │    sample_name    │ read_count │ base_count │
├───────────────────┼──────────────┼─────┼─────────────┼───────────────┼───────────┼────────────────┼─────────────────────┼─────────────────────────────┼───────┼───────────────────┼────────────┼────────────┤
│ NP_A_DNA_L001     │ NP_A         │ DNA │ 26613104    │ 191868        │ 21290514  │ Nasopharyngeal │ SARS-CoV-2 negative │ Asymptomatic-DNA-sequenced  │       │ NP_A_DNA_L001     │ 191868     │ 21290514   │
│ NP_A_cDNA_L001    │ NP_A         │ RNA │ 21402054    │ 290140        │ 32101115  │ Nasopharyngeal │ SARS-CoV-2 negative │ Asymptomatic-cDNA-sequenced │       │ NP_A_c

In [44]:
sqlite3 -batch sample_collab.db \
"select count(*) \
from ((SraRunTable sra inner join SampleOverviews spl on sra.host_subject_id=spl.sample_name) \
inner join ENARunTable ena on sra.Run=ena.run_accession) \
left join CtValues ct on patient_code=ct.Blinded_number \
where Ct>0;"

┌──────────┐
│ count(*) │
├──────────┤
│ 134      │
└──────────┘


In [45]:
sqlite3 -batch sample_collab.db "select * from bioinformaticians;"
sqlite3 -batch sample_collab.db "insert into bioinformaticians (username,firstname,lastname) values ('phingamp','Pascal','Hingamp');"
sqlite3 -batch sample_collab.db "select * from bioinformaticians;"

┌──────────┬───────────┬──────────┐
│ username │ firstname │ lastname │
├──────────┼───────────┼──────────┤
│ phingamp │ Pascal    │ Hingamp  │
└──────────┴───────────┴──────────┘


In [63]:
sqlite3 -batch sample_collab.db "select * from sample2bioinformatician;"
sqlite3 -batch sample_collab.db "insert into sample2bioinformatician (username,patient_code) values ('phingamp','NP_A');"
sqlite3 -batch sample_collab.db "insert into sample2bioinformatician (username,patient_code) values ('phingamp','NP_B');"
sqlite3 -batch sample_collab.db "insert into sample2bioinformatician (username,patient_code) values ('phingamp','NP_C');"
sqlite3 -batch sample_collab.db "insert into sample2bioinformatician (username,patient_code) values ('phingamp','NP_D');"
sqlite3 -batch sample_collab.db "insert into sample2bioinformatician (username,patient_code) values ('phingamp','NP_E');"
sqlite3 -batch sample_collab.db "select * from sample2bioinformatician;"

┌──────────┬──────────────┐
│ username │ patient_code │
├──────────┼──────────────┤
│ phingamp │ NP_A         │
│ phingamp │ NP_B         │
│ phingamp │ NP_C         │
│ phingamp │ NP_D         │
│ phingamp │ NP_E         │
└──────────┴──────────────┘


In [65]:
sqlite3 -batch sample_collab.db \
"select * from sample_annot spl left join (sample2bioinformatician s2b left join bioinformaticians b using(username)) using(patient_code) order by username desc limit 20;"

┌───────────────┬───────────────────┬──────────────┬─────┬────────────────┬─────────────────────┬─────────────────────────────┬───────┬─────────────┬────────────┬────────────┬──────────┬───────────┬──────────┐
│ run_accession │  host_subject_id  │ patient_code │ nuc │ host_body_site │ host_disease_status │   miscellaneous_parameter   │  Ct   │ total_reads │ read_count │ base_count │ username │ firstname │ lastname │
├───────────────┼───────────────────┼──────────────┼─────┼────────────────┼─────────────────────┼─────────────────────────────┼───────┼─────────────┼────────────┼────────────┼──────────┼───────────┼──────────┤
│ ERR6913174    │ NP_A_DNA_L001     │ NP_A         │ DNA │ Nasopharyngeal │ SARS-CoV-2 negative │ Asymptomatic-DNA-sequenced  │       │ 26613104    │ 191868     │ 21290514   │ phingamp │ Pascal    │ Hingamp  │
│ ERR6913175    │ NP_B_DNA_L001     │ NP_B         │ DNA │ Nasopharyngeal │ SARS-CoV-2 negative │ Asymptomatic-DNA-sequenced  │       │ 19924574    │ 399904    

In [66]:
sqlite3 -batch sample_collab.db \
"select count(*) from sample_annot spl left join (sample2bioinformatician s2b left join bioinformaticians b using(username)) using(patient_code) where username='phingamp';"

┌──────────┐
│ count(*) │
├──────────┤
│ 10       │
└──────────┘


# Don't forget to commit changes and leave this sandbox branch !

Change in the last git command below the branch name to your personal development branch name (should be your firstname)

In [ ]:
git add .
git commit -m "completed sandbox DB build"
git checkout ==your_firstname==